# Multi-Agent Researcher with MCP Support

This notebook demonstrates how to use the multi-agent research system with Model Context Protocol (MCP) support. It follows the same patterns as the original multi_agent.ipynb but adds MCP integration.

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src


In [2]:
import uuid 
import os
import open_deep_research   
print(open_deep_research.__version__) 
from IPython.display import Image, display, Markdown
from langgraph.checkpoint.memory import MemorySaver
import logging

# Set up logging for better visibility
logging.basicConfig(level=logging.INFO)

0.0.15


## Configure MCP Server

We'll use the example MCP server that provides patient data.

In [3]:
# Get the path to the example_mcp_server.py 
import pathlib
repo_root = pathlib.Path(".").resolve()
example_server_path = str(repo_root / "open_deep_research" / "example_mcp_server.py")

# Check if file exists
print(f"Example MCP server path: {example_server_path}")
print(f"File exists: {pathlib.Path(example_server_path).exists()}")

# Try alternate locations if not found
if not pathlib.Path(example_server_path).exists():
    alternate_path = str(repo_root / "open_deep_research" / "example_mcp_server.py")
    print(f"Trying alternate path: {alternate_path}")
    print(f"Alternate path exists: {pathlib.Path(alternate_path).exists()}")
    if pathlib.Path(alternate_path).exists():
        example_server_path = alternate_path

# Define MCP server configuration
mcp_servers_config = {
    "patient_data": {
        "transport": "stdio",
        "command": "python",
        "args": [example_server_path],
    }
}

Example MCP server path: /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
File exists: True


## Build the Graph with MCP Support

Instead of directly using supervisor_g.compile(), we'll use the build_graph_with_mcp function.

In [ ]:
# Import the build_graph_with_mcp function
from open_deep_research.multi_agent_mcp import build_graph_with_mcp, supervisor_builder

# Create a MemorySaver for checkpointing
checkpointer = MemorySaver()

# Create base configuration with proper model specifications
base_config = {
    "thread_id": str(uuid.uuid4()),
    "search_api": "tavily",
    # "supervisor_model": "anthropic:claude-3-5-sonnet-latest",  # Use appropriate model
    # "researcher_model": "anthropic:claude-3-5-sonnet-latest",  # Use appropriate model
    "mcp_servers": mcp_servers_config,
    "_mcp_servers_config": mcp_servers_config,
}

# This is a notebook-specific adaptation that combines build_graph_with_mcp with checkpointer
async def build_graph_with_checkpointer(cfg):
    # First get the MCP configuration but don't create the manager yet (like in build_graph_with_mcp)
    from open_deep_research.configuration import Configuration
    
    cfg = cfg or {}
    configurable = Configuration.from_runnable_config(cfg)
    mcp_servers = configurable.mcp_servers
    
    # Store MCP servers config in a special key (same as build_graph_with_mcp)
    if mcp_servers:
        print(f"MCP servers configured: {list(mcp_servers.keys())}")
        if "configurable" not in cfg:
            cfg["configurable"] = {}
        cfg["configurable"]["_mcp_servers_config"] = mcp_servers
    
    # Now compile with checkpointer (different from build_graph_with_mcp)
    g = supervisor_builder.compile(name="research_team", checkpointer=checkpointer)
    
    # Apply config after compilation
    g = g.with_config(cfg)
    
    # Return in the same format as build_graph_with_mcp
    return g, None, cfg

# Build the graph using our adapted function
agent, _, thread_config = await build_graph_with_checkpointer(base_config)

MCP servers configured: ['patient_data']


## Visualize the Graph Structure

This shows how supervisor and research agents are connected in the workflow.

In [ ]:
display(Image(agent.get_graph(xray=1).draw_mermaid_png(max_retries=5)))

## Run the Multi-Agent System

We'll use a focused query that leverages the patient data from the MCP server.

In [ ]:
# Define the research topic as a user message
msg = [{"role": "user", "content": "Check what we know about patient with id P001 and conduct research on the latest treatment approaches and management strategies for these conditions."}]

# Create a properly structured config dictionary for invoking the agent
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Run the multi-agent workflow with the properly structured configuration
response = await agent.ainvoke({"messages": msg}, config=access_config)

INFO:open_deep_research.multi_agent_new:Initializing MCP manager for graph session
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    Dictionary containing vital measurements including:
    - heart_rate: Beats per minute
    - blood_pressure: Systolic/diastolic in mmHg
    - temperature: Body te

## View the Supervisor's Response

In [ ]:
# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
    }
}

# Then use this properly structured config to access the state
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

================================== Ai Message ==================================

Here is a summary of what we know about patient P001:

**Diagnosed Conditions:**
- Type 2 Diabetes Mellitus (moderate severity, managed, diagnosed July 2023)
- Essential Hypertension (mild severity, active, diagnosed March 2022)

**Current Vitals (stable):**
- Heart rate: 72 bpm
- Blood pressure: 120/80 mmHg
- Temperature: 36.8°C
- Respiratory rate: 16 bpm
- Oxygen saturation: 98%

**Upcoming Check:**
- HbA1c Test scheduled for May 10, 2025 (requires 8-hour fasting)

**Latest Treatment and Management Strategies (as of 2024–2025):**
- The American Diabetes Association’s 2025 Standards of Care emphasize integrated, evidence-based approaches to both Type 2 Diabetes and associated conditions like hypertension.
- Focus areas include personalized nutrition, use of weight-loss medications, and increased adoption of diabetes technology (such as continuous glucose monitoring).
- There is growing emphasis on stress

## Add Clarification if Needed

If the supervisor asks for clarification, we can add more details.

In [ ]:
# Create a follow-up message with your clarification
follow_up_msg = [
    {"role": "user", "content": "Focus on the latest integrated management approaches and patient self-management. The report is for patient self-education. Adjust narrative accordingly. Target audience: average."},
]

# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
        "_mcp_servers_config": base_config["_mcp_servers_config"],
    }
}

# Run the multi-agent workflow again with your clarification
response = await agent.ainvoke({"messages": follow_up_msg}, config=access_config)

# Check the response
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

INFO:open_deep_research.multi_agent_new:Initializing MCP manager for graph session
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    Dictionary containing vital measurements including:
    - heart_rate: Beats per minute
    - blood_pressure: Systolic/diastolic in mmHg
    - temperature: Body te

================================== Ai Message ==================================

Thank you for clarifying! Here is a synthesized summary of what we’ve learned so far and how it guides the next step:

**Patient Profile Recap:**  
P001 has moderate, managed type 2 diabetes and mild, active hypertension. Vitals are stable, and the patient has an upcoming routine HbA1c test.  

**Latest Treatment Landscape:**  
2025 guidelines (ADA, IDF, clinical resources) recommend integrating care for diabetes and hypertension—focusing on nutrition, lifestyle, newer medications, technology (such as glucose monitoring), and minimizing cardiovascular risk. There’s strong emphasis on patient self-management, routine monitoring, and understanding the importance of blood pressure and blood sugar control together.

**Your Clarifications:**  
- Focus: Latest integrated management for both conditions; explain patient self-management.
- Tone/Depth: For patient self-education; clear, practical, non-technical lan

In [8]:
# Create a follow-up message with your clarification
confirmation_msg = [
    {"role": "user", "content": "I confirm the scope. Adjust narrative for average audience. Focus on personal lifestyle/self-management day in the life."}
]

# Create a properly structured config dictionary for state access
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
        "_mcp_servers_config": base_config["_mcp_servers_config"],
    }
}

# Run the multi-agent workflow again with your clarification
response = await agent.ainvoke({"messages": confirmation_msg}, config=access_config)

# Check the response
messages = agent.get_state(access_config).values['messages']
messages[-1].pretty_print()

INFO:open_deep_research.multi_agent_new:Initializing MCP manager for graph session
INFO:open_deep_research.mcp_integration:Creating MCP manager with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Initializing MCP client with servers: ['patient_data']
INFO:open_deep_research.mcp_integration:Server patient_data config: transport=stdio
INFO:open_deep_research.mcp_integration:  Command: python /Users/bartolli/Projects/playgound/open_deep_research_05/open_deep_research/src/open_deep_research/example_mcp_server.py
INFO:open_deep_research.mcp_integration:Successfully loaded 3 tools from MCP servers
INFO:open_deep_research.mcp_integration:Tool 1: patient_vitals - 
Use this tool to retrieve current vital signs for a patient for your research.

Args:
    patient_id: Unique identifier for the patient
    
Returns:
    Dictionary containing vital measurements including:
    - heart_rate: Beats per minute
    - blood_pressure: Systolic/diastolic in mmHg
    - temperature: Body te

ValueError: Supervisor LLM failed due to unresolved tool calls: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_TV8A5ZDeZAS3zmmrI5oRDThN", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

## View the Final Report

Once all sections are completed, introduction and conclusion are added, the final report will be available.

In [ ]:
# Create the properly structured access config
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
        "_mcp_servers_config": base_config.get("_mcp_servers_config", base_config["mcp_servers"]),
    }
}

# Get the current state
state = agent.get_state(access_config).values

# Check if the final report is available
if 'final_report' in state and state['final_report']:
    print("✅ Final report is complete:")
    display(Markdown(state['final_report']))
else:
    print("⏳ Report is still being generated. Showing available content so far:")
    
    # Construct a partial report from available components
    report_parts = []
    
    # Add introduction if available
    if 'final_report' in state and state['final_report']:
        report_parts.append(state['final_report'])  # This might contain just the intro
    
    # Add completed sections if available
    if 'completed_sections' in state and state['completed_sections']:
        print(f"- {len(state['completed_sections'])} sections completed")
        for section in state['completed_sections']:
            report_parts.append(section.content)
    
    # Display what we have so far
    if report_parts:
        display(Markdown("\n\n".join(report_parts)))
    else:
        print("No content available yet.")
        
    # Show message count for debugging
    print(f"\nCurrent message count: {len(state['messages'])}")

## Optional: Continue the Research Process

If the report isn't complete, you can continue with further interactions.

In [ ]:
# Optional: Request elaboration on a specific section or topic
# Uncomment and modify when you want to explore a specific area in more detail

# Create a list of possible section names if they exist in the state
access_config = {
    "configurable": {
        "thread_id": base_config["thread_id"],
        "mcp_servers": base_config["mcp_servers"],
        "_mcp_servers_config": base_config.get("_mcp_servers_config", base_config["mcp_servers"]),
    }
}

# Check if there are completed sections to reference
if 'completed_sections' in agent.get_state(access_config).values:
    section_names = [section.name for section in agent.get_state(access_config).values['completed_sections']]
    print("Available sections that could be elaborated:")
    for i, name in enumerate(section_names):
        print(f"{i+1}. {name}")



In [ ]:
# If you want to elaborate on a specific section, uncomment and modify the following lines
# Choose a specific section to elaborate on

# Choose a specific topic to elaborate on
elaborate_topic = "Lifestyle and Behavioral Interventions: Evidence-Based Approaches and Emerging Trends"

# Request elaboration within the current thread
elaborate_msg = [
    {"role": "user", "content": f"Now that we have a good understanding of Patient P001's condition from the report, I'd like more detailed information specifically about '{elaborate_topic}'. Please create a focused mini-report that goes deeper into this topic, including the most recent clinical evidence, medication classes, dosing considerations, and monitoring recommendations. Focus specifically on how this applies to Patient P001's clinical profile of moderate Type 2 Diabetes (HbA1c 7.8%) and mild Hypertension (BP 135/88)."}
]

# Run the process for the elaboration using the SAME thread configuration
elaborate_response = await agent.ainvoke({"messages": elaborate_msg}, config=access_config)

# Check the response
elaborate_messages = agent.get_state(access_config).values['messages']
elaborate_messages[-1].pretty_print()